In [ ]:
# Import boto and smart_open to import data from S3 to Kafka for ingestion
import botocore
import boto3
import threading, logging, time
from kafka import KafkaProducer
import smart_open

# Kafka producer
class Producer(threading.Thread):
    daemon = True

    # Send data from S3 to Kafka queue
    def run(self):
        producer = KafkaProducer(bootstrap_servers='localhost:port')
        bucket_name = 's3bucketnm'
        bucket = self.__get_s3_bucket__(bucket_name)
        print("startIngestion")
        for i in range(1,2):
            for line in  smart_open.smart_open("s3://s3bucketnm/stream1.tsv"):
                producer.send('topic_click', line)
        print("doneIngestion")
                
    # Access S3 bucket
    def __get_s3_bucket__(self, bucket_name):
        s3 = boto3.resource('s3')
        try:
            s3.meta.client.head_bucket(Bucket=bucket_name)
        except botocore.exceptions.ClientError as e:
            # If a client error is thrown, check that it was a 404 error.
            # If it was a 404 error, then the bucket does not exist.
            error_code = int(e.response['Error']['Code'])
            if error_code == 404:
                exists = False
                print (e.response['404 Error: bucket not found'])
            else:
                print (e.response['Error'])
            return None
        else:
            return s3.Bucket(bucket_name)


if __name__ == "__main__":
    Producer().start()
    while True:
        time.sleep(10)


before
